In [1]:
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error
import mlflow
import optuna
import os
import sys
from utils import train_test_split
from preprocessing.preprocessing import *
import time
from datetime import datetime, timezone

# import warnings
# warnings.filterwarnings("ignore", category=UserWarning)

# Global variables

In [2]:
forecast_horizon = 48
n_lags = 48
model_name = "lightgbm"
preprocessing_version = 2
config_version = 1
config_dir_path = "./configuration_files"
study_path = "./optuna_studies"
random_state = 0

In [3]:
# forecast_horizon = 24
# n_lags = 48
# test_window = 24*60 # in hours
# preprocessing_version = 1 # preprocessing version
# study_version = 1 # Optuna study to take hyperparameters from
# random_state = 0

# Read the Data

In [4]:
df = pd.read_csv("./data/consumption.csv")
df["datetime"] = pd.to_datetime(df["datetime"])
df.head()

,datetime,prediction_unit_id,consumption
0,2021-09-01,0,96.590
1,2021-09-01,1,17.314
2,2021-09-01,2,656.859
3,2021-09-01,3,59.000
4,2021-09-01,4,501.760


In [5]:
df.shape

(1009176, 3)

# Train/Test split

In [6]:
# taking the last 60 days of each unit for test
df_train, df_test = train_test_split(df, test_window=24 * 60)

In [7]:
assert df.shape[0] == df_train.shape[0] + df_test.shape[0]
assert df.shape[1] == df_train.shape[1] == df_test.shape[1]

In [8]:
test_size = df_test.shape[0] / (df.shape[0])
print(f"test set : {round(test_size*100, 2)}% of the data set")

test set : 9.85% of the data set


# Preprocessing

In [9]:
print(f"Using preprocessing version {preprocessing_version}")
preprocessing = vars()[f"preprocessing_{preprocessing_version}"]

Using preprocessing version 2


In [10]:
X_train, y_train = preprocessing(df_train)
print(f"X_train shape : {X_train.shape}")
print(f"y_train shape : {y_train.shape}")
X_train.head()

/Users/gabriel/Documents/Git/End-to-end MLOps for Time Series/.venv/lib/python3.10/site-packages/mlforecast/core.py:455: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[feat_name] = feat_vals[restore_idxs]
/Users/gabriel/Documents/Git/End-to-end MLOps for Time Series/.venv/lib/python3.10/site-packages/mlforecast/core.py:455: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[feat_name] = feat_vals[restore_idxs]


X_train shape : (854079, 99)
y_train shape : (854079,)


,lag48,lag49,lag50,lag51,lag52,lag53,lag54,lag55,lag56,lag57,...,rolling_mean_lag68_window_size24,expanding_mean_lag69,rolling_mean_lag69_window_size24,expanding_mean_lag70,rolling_mean_lag70_window_size24,expanding_mean_lag71,rolling_mean_lag71_window_size24,month,dayofweek,hour
5795,120.540,134.986,150.412,152.763,136.130,121.033,80.621,43.428,46.840,43.671,...,82.505417,82.612111,81.860500,81.756654,81.308000,81.145600,80.502083,9,5,23
5856,107.129,120.540,134.986,150.412,152.763,136.130,121.033,80.621,43.428,46.840,...,83.241833,83.355714,82.505417,82.612111,81.860500,81.756654,81.308000,9,6,0
5917,81.920,107.129,120.540,134.986,150.412,152.763,136.130,121.033,80.621,43.428,...,83.893958,84.131655,83.241833,83.355714,82.505417,82.612111,81.860500,9,6,1
5978,96.193,81.920,107.129,120.540,134.986,150.412,152.763,136.130,121.033,80.621,...,84.539375,84.841667,83.893958,84.131655,83.241833,83.355714,82.505417,9,6,2
6039,94.536,96.193,81.920,107.129,120.540,134.986,150.412,152.763,136.130,121.033,...,84.552333,85.716806,84.539375,84.841667,83.893958,84.131655,83.241833,9,6,3


In [11]:
X_test, y_test = preprocessing(df_test)
print(f"X_test shape : {X_test.shape}")
print(f"y_test shape : {y_test.shape}")
X_test.head()

X_test shape : (84735, 99)
y_test shape : (84735,)


/Users/gabriel/Documents/Git/End-to-end MLOps for Time Series/.venv/lib/python3.10/site-packages/mlforecast/core.py:455: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[feat_name] = feat_vals[restore_idxs]
/Users/gabriel/Documents/Git/End-to-end MLOps for Time Series/.venv/lib/python3.10/site-packages/mlforecast/core.py:455: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[feat_name] = feat_vals[restore_idxs]


,lag48,lag49,lag50,lag51,lag52,lag53,lag54,lag55,lag56,lag57,...,rolling_mean_lag68_window_size24,expanding_mean_lag69,rolling_mean_lag69_window_size24,expanding_mean_lag70,rolling_mean_lag70_window_size24,expanding_mean_lag71,rolling_mean_lag71_window_size24,month,dayofweek,hour
920263,1057.285,1107.518,1177.874,1114.973,860.687,624.847,382.365,246.753,193.661,160.944,...,722.741458,747.325111,723.395542,740.683269,723.535208,734.978880,725.296292,4,2,23
920328,1055.621,1057.285,1107.518,1177.874,1114.973,860.687,624.847,382.365,246.753,193.661,...,723.696750,753.981429,722.741458,747.325111,723.395542,740.683269,723.535208,4,3,0
920393,999.628,1055.621,1057.285,1107.518,1177.874,1114.973,860.687,624.847,382.365,246.753,...,724.803458,761.656966,723.696750,753.981429,722.741458,747.325111,723.395542,4,3,1
920458,1001.917,999.628,1055.621,1057.285,1107.518,1177.874,1114.973,860.687,624.847,382.365,...,726.249667,769.130867,724.803458,761.656966,723.696750,753.981429,722.741458,4,3,2
920523,1014.902,1001.917,999.628,1055.621,1057.285,1107.518,1177.874,1114.973,860.687,624.847,...,731.313625,778.516258,726.249667,769.130867,724.803458,761.656966,723.696750,4,3,3


# Train a vanilla LightGBM

In [12]:
model = LGBMRegressor(random_state=random_state)
start = time.time()
model.fit(X_train, y_train)
end = time.time()

# metrics
y_fit = model.predict(X_train)
y_pred = model.predict(X_test)
train_mae = mean_absolute_error(y_train, y_fit)
test_mae = mean_absolute_error(y_test, y_pred)
training_duration = end - start
metrics = {"train_mae": train_mae, "test_mae": test_mae, "training_duration": training_duration}
metrics

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.038800 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 24524
[LightGBM] [Info] Number of data points in the train set: 854079, number of used features: 99
[LightGBM] [Info] Start training from score 485.532565


{'train_mae': np.float64(66.30995098429447),
 'test_mae': np.float64(91.88798810682138),
 'training_duration': 5.6921632289886475}

# Setting up the MLFlow experiment

From a terminal, run:
```
mlflow server --host 127.0.0.1 --port 5000
```

In [13]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [14]:
# Set the current active MLflow experiment
experiment_name = f"Enefit {model_name} Preprocessing {preprocessing_version}"
mlflow.set_experiment(experiment_name=experiment_name)

<Experiment: artifact_location='mlflow-artifacts:/847028692385838956', creation_time=1737798748143, experiment_id='847028692385838956', last_update_time=1737798748143, lifecycle_stage='active', name='Enefit lightgbm Preprocessing 2', tags={}>

In [15]:
experiment = mlflow.get_experiment_by_name(experiment_name)
print(f"ID for experiment '{experiment_name}': {experiment.experiment_id}")

ID for experiment 'Enefit lightgbm Preprocessing 2': 847028692385838956


# Log the study's best model

In [16]:
study_name = (
    f"{model_name}_preprocessing{preprocessing_version}_config{config_version}"
)
storage_path = "sqlite:///{}/{}.db".format(study_path, study_name)
print(f"Study name : {study_name}")
study = optuna.create_study(study_name=study_name, storage=storage_path, load_if_exists=True)

Study name : lightgbm_preprocessing2_config1


[I 2025-01-25 14:28:51,931] Using an existing study with name 'lightgbm_preprocessing2_config1' instead of creating a new one.


In [ ]:
# run metadata
utc_datetime = datetime.now(timezone.utc).strftime("%Y%m%dT%H%M%SUTC")
run_name = f"best_model_{study_name}_{utc_datetime}"

# training
params = study.best_params
model = LGBMRegressor(**params, random_state=random_state)
start = time.time()
model.fit(X_train, y_train)
end = time.time()

# metrics
y_fit = model.predict(X_train)
y_pred = model.predict(X_test)
train_mae = mean_absolute_error(y_train, y_fit)
test_mae = mean_absolute_error(y_test, y_pred)
training_duration = end - start
metrics = {"train_mae": train_mae, "test_mae": test_mae, "training_duration": training_duration}

with mlflow.start_run(run_name=run_name) as run:
    # Log the parameters used for the model fit
    mlflow.log_params(params)

    # Log the error metrics that were calculated during validation
    mlflow.log_metrics(metrics)

    # Log an instance of the trained model for later use
    mlflow.lightgbm.log_model(
        lgb_model=model, input_example=X_train.iloc[:1], artifact_path=model_name
    )

/Users/gabriel/Documents/Git/End-to-end MLOps for Time Series/.venv/lib/python3.10/site-packages/lightgbm/engine.py:204: UserWarning: Found `num_trees` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] feature_fraction is set=0.592949529294766, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.592949529294766
[LightGBM] [Warning] num_iterations is set=10, num_trees=10 will be ignored. Current value: num_iterations=10
[LightGBM] [Warning] feature_fraction is set=0.592949529294766, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.592949529294766
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.020896 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 24524
[LightGBM] [Info] Number of data points in the train set: 854079, number of used features: 99
[LightGBM] [Info] Start training from score 485.532565
[LightGBM] [Warning] feature_fraction is set=0.592949529294766, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.592949529294766
[LightGBM] [Warning] fea

/Users/gabriel/Documents/Git/End-to-end MLOps for Time Series/.venv/lib/python3.10/site-packages/mlflow/types/utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


[LightGBM] [Warning] feature_fraction is set=0.592949529294766, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.592949529294766


2025/01/25 14:29:01 INFO mlflow.tracking._tracking_service.client: 🏃 View run best_model_lightgbm_preprocessing2_config1_20250125T102851UTC at: http://127.0.0.1:5000/#/experiments/847028692385838956/runs/abc16b1ed5ea4721920091e379d81db6.
2025/01/25 14:29:01 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/847028692385838956.


[LightGBM] [Warning] feature_fraction is set=0.592949529294766, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.592949529294766
